In [1]:
import pandas as pd

In [14]:
l1data = pd.read_excel("../../../../Outcome Measurement Data/MALARIA/Solicitud 0858-2019 Malaria L1 del 2017 al 2019.xlsx", skiprows=3)

In [16]:
l1data.columns

Index(['departamento', 'municipio', 'priorizado', 'area_salud',
       'distrito_salud', 'servicio_salud', 'tipo_servicio', 'laboratorio_id',
       'anio_registro', 'mes_registro', 'mes', 'muestra', 'gota_gruesa',
       'prueba_rapida', 'resultado_prueba_rapida', 'Código paciente', 'cui',
       'sexo', 'pueblo', 'comunidad_linguistica', 'fecha_nacimiento',
       'discapacidad', 'orientacion_sexual', 'escolaridad',
       'profesion_oficio_condicion', 'agricola_migrante', 'embarazada',
       'notificante', 'no_e1', 'fecha_inicio_sintomas', 'fecha_toma_muestras',
       'fecha_diagnostico', 'fecha_inicio_tratamiento',
       'fecha_finalizacion_tratamiento', 'completo_tratamiento',
       'causa_suspension', 'cantidad_entregada_cloroquina',
       'cantidad_entregada_primaquina', 'resultado', 'densidad_parasitaria',
       'gametocitos'],
      dtype='object')

In [20]:
l1data.groupby(["anio_registro", "notificante"]).apply(len).unstack(0)

anio_registro,2017.0,2018.0,2019.0
notificante,,,
Centro de Salud (C/S),237.0,1229.0,438.0
Centro microcoscopía,42.0,227.0,79.0
Colaborador Voluntario (CV),295.0,1117.0,426.0
Cruz Roja,NaN,1.0,NaN
Hospital,25.0,145.0,25.0
Hospital Privado,1.0,3.0,1.0
IGSS,6.0,15.0,1.0
Otro,62.0,51.0,32.0
Polivalente y/o polifuncional,89.0,363.0,173.0


In [47]:
l1data["colvol"] = (l1data.notificante == "Colaborador Voluntario (CV)").map(lambda x: "COLVOL" if x else "Otro")

In [53]:
l1data.groupby(["anio_registro", "colvol", "departamento"]).apply(len).unstack([0,1]).fillna(0)

anio_registro  2017.0        2018.0        2019.0       
colvol         COLVOL   Otro COLVOL   Otro COLVOL   Otro
departamento                                            
EL PETEN          2.0   15.0   15.0  132.0    3.0   31.0
ESCUINTLA       240.0  214.0  805.0  631.0  358.0  385.0
SUCHITEPEQUEZ    53.0   58.0   38.0   83.0    5.0   22.0
ALTA VERAPAZ      0.0   21.0  255.0  425.0   59.0   65.0
CHIMALTENANGO     0.0  125.0    0.0  415.0    0.0  248.0
GUATEMALA         0.0    4.0    0.0    0.0    0.0    0.0
IZABAL            0.0    2.0    1.0   27.0    0.0   17.0
QUICHE            0.0    1.0    1.0  102.0    1.0    2.0
RETALHULEU        0.0    4.0    2.0   16.0    0.0    2.0
SAN MARCOS        0.0    1.0    0.0    0.0    0.0    0.0
SOLOLA            0.0   52.0    0.0   17.0    0.0    0.0
ZACAPA            0.0    1.0    0.0    4.0    0.0    1.0
HUEHUETENANGO     0.0    0.0    0.0    4.0    0.0    2.0
JALAPA            0.0    0.0    0.0  293.0    0.0   10.0
JUTIAPA           0.0    0.0    0.0    1.0    0.0    0.0
QUETZALTENANGO    0.0    0.0    0.0    1.0    0.0    0.0

In [48]:
cols = ['sexo', 'pueblo', 'comunidad_linguistica', 
       'discapacidad', 'orientacion_sexual', 'escolaridad',
       'profesion_oficio_condicion', 'agricola_migrante', 'embarazada']

colvolsdata = pd.concat([l1data.groupby([
    "anio_registro", "colvol", x]).apply(len).unstack([0,1]).rename(index = lambda y: x + "-" + y if type(y) == str else y) \
                         for x in cols]) #.rename(index=lambda y: (y[0], x + "-" + y[1])) for x in cols])

In [50]:
colvolsdata.fillna(0)

anio_registro                          2017.0         2018.0         2019.0  \
colvol                                 COLVOL   Otro  COLVOL    Otro COLVOL   
sexo-F                                  128.0  337.0   537.0  1459.0  194.0   
sexo-M                                  167.0  161.0   580.0   692.0  232.0   
pueblo-Maya                               1.0  196.0   260.0   974.0   62.0   
pueblo-Mestizo-Ladino                   258.0  271.0   840.0  1106.0  357.0   
pueblo-No Indica                         36.0   31.0    17.0    70.0    7.0   
pueblo-Otro                               0.0    0.0     0.0     1.0    0.0   
comunidad_linguistica-Ixil                0.0    0.0     0.0     1.0    0.0   
comunidad_linguistica-Jakalteka           0.0    0.0     0.0     1.0    0.0   
comunidad_linguistica-K'iche'             0.0    1.0     0.0    33.0    1.0   
comunidad_linguistica-Kaqchikel           0.0  150.0     0.0   407.0    0.0   
comunidad_linguistica-Mam                 0.0    0.0     0.0     2.0    0.0   
comunidad_linguistica-Mopan               0.0    1.0     0.0     0.0    0.0   
comunidad_linguistica-No indica         294.0  322.0   860.0  1211.0  364.0   
comunidad_linguistica-Q'anjob'al          0.0    0.0     0.0     1.0    0.0   
comunidad_linguistica-Q'eqchi'            1.0   23.0   257.0   437.0   61.0   
comunidad_linguistica-Sakapulteko         0.0    0.0     0.0    58.0    0.0   
comunidad_linguistica-Tz'utujil           0.0    1.0     0.0     0.0    0.0   
discapacidad-                           295.0  498.0  1117.0  2151.0  426.0   
orientacion_sexual-Bisexual               0.0    0.0     0.0     1.0    0.0   
orientacion_sexual-Heterosexual           1.0   59.0     2.0   215.0    3.0   
orientacion_sexual-No aplica            294.0  439.0  1114.0  1933.0  422.0   
orientacion_sexual-Otro                   0.0    0.0     0.0     2.0    1.0   
orientacion_sexual-Trans                  0.0    0.0     1.0     0.0    0.0   
escolaridad-Básicos                       0.0    2.0     1.0    11.0    0.0   
escolaridad-Diversificado                 0.0    1.0     0.0     5.0    0.0   
escolaridad-Ninguno                       0.0    0.0     0.0    13.0    0.0   
escolaridad-No aplica                   294.0  444.0  1095.0  1999.0  420.0   
escolaridad-No indica                     0.0   47.0    13.0    52.0    5.0   
escolaridad-Otro                          0.0    0.0     0.0     1.0    0.0   
escolaridad-Pre Primaria                  0.0    0.0     0.0     2.0    0.0   
escolaridad-Primaria                      1.0    4.0     8.0    68.0    1.0   
profesion_oficio_condicion-Ama de Casa    0.0    1.0     0.0    85.0    0.0   
profesion_oficio_condicion-No aplica    294.0  494.0  1116.0  2061.0  425.0   
profesion_oficio_condicion-Otros          1.0    3.0     1.0     5.0    1.0   
agricola_migrante-SIN DATO              282.0  484.0  1115.0  2132.0  426.0   
agricola_migrante-Sí                     13.0   14.0     2.0    19.0    0.0   
embarazada-                             290.0  330.0  1112.0  1624.0  423.0   
embarazada-Sí                             5.0  168.0     5.0   527.0    3.0   

anio_registro                                  
colvol                                   Otro  
sexo-F                                  516.0  
sexo-M                                  269.0  
pueblo-Maya                             332.0  
pueblo-Mestizo-Ladino                   438.0  
pueblo-No Indica                         15.0  
pueblo-Otro                               0.0  
comunidad_linguistica-Ixil                0.0  
comunidad_linguistica-Jakalteka           0.0  
comunidad_linguistica-K'iche'             0.0  
comunidad_linguistica-Kaqchikel         244.0  
comunidad_linguistica-Mam                 1.0  
comunidad_linguistica-Mopan               0.0  
comunidad_linguistica-No indica         459.0  
comunidad_linguistica-Q'anjob'al          0.0  
comunidad_linguistica-Q'eqchi'           81.0  
comunidad_linguistica-S